In [1]:
# Compute Call and Put Probabilities
# Compute the Return on Margin per year
# Compute how far the Strike Price is from Yearly High (for Call) and Low (for Put) in %
# Filter out Last Traded Price > BS
# Filter out only those that have Open Interests
# Reverse Sort the result on RoM * Probability

# Fish out Probabilities > 92%

# Read the file
import pandas as pd
df = pd.read_excel('NSE.xlsx')
df.head()

# Compute Call and Put Probabilities
df['pProb'] = 1-df.cProb

# Compute Return on Margins
df['cRoM'] = df.Call_LTP/(df.MarginRate*df.Underlying)*365/df.DTE  # Call Return on Margin
df['pRoM'] = df.Put_LTP/(df.MarginRate*df.Underlying)*365/df.DTE  # Put Return on Margin

# How close Price is from Hi52 (for calls), and Lo52 (for Puts)
# Negative values for 52Delta is better - as they are beyond Hi/Lo52
df['Delta52'] = (df.Hi52-df.Strike)/df.Strike                                 # for Calls
df.loc[df.Strike < df.Underlying, "Delta52"] = (df.Strike-df.Lo52)/df.Strike  # for Puts

# Filter out LTP > BS and there is some OI on these
df = df[((df.Call_LTP > df.BSCall) | (df.Put_LTP > df.BSPut)) &
        ((df.Call_OI > 0) | (df.Put_OI > 0))]

# Filter out Call_LTP = 0 for Strike Price > Underlying and 
#            Put_LTP = 0 for Strike Price < Underlying
df = df[((df.Call_LTP > 0) & (df.Strike > df.Underlying)) |
   ((df.Put_LTP > 0) & (df.Strike < df.Underlying))]

# Calculate the result on RoM * Probability and reverse sort on it
df['RoMPoP'] = df.cProb * df.cRoM                                 # For Palls
df.loc[df.Strike < df.Underlying, "RoMPoP"] = df.pProb * df.pRoM  # For Puts
df = df.sort_values(by=['RoMPoP'], ascending=False).reset_index(drop=True)

# Identify the Calls and Puts
c = df[df.cProb>0.98]
p = df[df.pProb>0.95]

# Get only the negative Delat52s
calls = c[c.Delta52<0]
puts = p[p.Delta52<0]

In [2]:
calls[['Symbol', 'Strike', 'Expiry', 'Call_LTP', 'Underlying', 'Hi52', 'Delta52', 'RoMPoP', 'cRoM', 'cProb', 'Call_OI', 'Call_AskPrice']]
puts[['Symbol', 'Strike', 'Expiry', 'Put_LTP', 'Underlying', 'Lo52', 'Delta52', 'RoMPoP', 'pRoM', 'pProb', 'Put_OI', 'Put_AskPrice']]

,Symbol,Strike,Expiry,Put_LTP,Underlying,Lo52,Delta52,RoMPoP,pRoM,pProb,Put_OI,Put_AskPrice
783,RECLTD,130.0,22FEB2018,1.50,148.45,139.35,-0.071923,1.408242,1.475244,0.954582,378000,1.45
997,BALRAMCHIN,95.0,22FEB2018,1.00,113.45,109.05,-0.147895,1.115334,1.163993,0.958197,14000,1.90
1021,HDIL,40.0,22FEB2018,0.50,53.30,47.00,-0.175000,1.088741,1.094634,0.994616,27000,0.60
1036,PFC,100.0,22FEB2018,0.85,113.40,111.90,-0.119000,1.069339,1.094356,0.977140,306000,0.80
1090,SUZLON,10.0,22FEB2018,0.10,13.90,12.80,-0.280000,1.024079,1.024142,0.999939,525000,0.10
1192,BHEL,80.0,22FEB2018,0.65,94.90,80.87,-0.010875,0.901503,0.923191,0.976508,262500,0.65
1286,HAVELLS,310.0,22FEB2018,3.00,514.80,396.25,-0.278226,0.800240,0.800241,1.000000,2000,3.75
1293,TATAMOTORS,340.0,22FEB2018,2.15,386.75,358.50,-0.054412,0.795860,0.811635,0.980564,259500,2.20
1357,IOC,360.0,22FEB2018,2.10,411.55,363.35,-0.009306,0.725129,0.744988,0.973342,255000,2.15
1401,RELCAPITAL,300.0,22FEB2018,3.00,446.70,397.00,-0.323333,0.678387,0.679033,0.999048,750,2.90


In [3]:
writer = pd.ExcelWriter('puts_calls.xlsx')
puts.to_excel(writer, 'Puts', index=False , freeze_panes=(1,1))
calls.to_excel(writer, 'Calls', index=False, freeze_panes=(1,1))
writer.save()